<a href="https://colab.research.google.com/github/HpDiniz/Leitor-de-PDF/blob/main/PDF_Extractor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pdfminer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
import io
import re
import os
from io import StringIO
from google.colab import files
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from pdfminer.converter import TextConverter
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter

In [3]:
file_objects = files.upload()

Saving Extrato XP - Cliente 01.pdf to Extrato XP - Cliente 01 (5).pdf


In [8]:
# Converte o conteúdo do PDF para texto
def convert_pdf_to_txt(path):
    
    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, laparams=laparams)
    fp = open(path, 'rb')
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    password = ""
    maxpages = 0
    caching = True
    pagenos=set()

    for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages, password=password,caching=caching, check_extractable=True):
        interpreter.process_page(page)

    text = retstr.getvalue()

    fp.close()
    device.close()
    retstr.close()

    return text

In [9]:
for file in list(file_objects.keys()):

    file_name, file_extension = os.path.splitext(file)

    if file_extension != '.pdf':
        continue

    # Lê o conteúdo do arquivo PDF
    pdf_text = convert_pdf_to_txt(file)

    # Exibe o conteúdo
    print(pdf_text)

Extrato de Cotista

Consolidado

Movimentação de 15/05/2022 a 11/11/2022

Nome do Cliente
Endereço

CEP: 02440-00

POSIÇÃO CONSOLIDADA

Fundo

Cota

Quantidade

Valor Aplicado

Valor Bruto

IR

SulAmérica Inflatie FI Renda Fixa Longo Prazo

XP Referenciado FI Referenciado DI CP

XPA MM II FIC FIM

XPA RV FIC FIM

XPA Vértice Crédito FIM CP

49,03907260

2,55168782

1,48495549

1,51471586

1,31704969

485,46020696

9.607,86926679

206.015,91600648

86.244,99799744

62.558,82325043

21.434,59

23.303,62

231.165,15

119.600,00

67.664,32

23.806,52

24.516,28

305.924,47

130.636,67

82.393,08

13,80

272,84

2.095,63

1.655,50

816,74

Emissão: 11/11/2022

Código: 3Nxxx7

IOF

0,00

0,00

0,00

0,00

0,00

Valor Liquido

23.792,72

24.243,44

303.828,84

128.981,17

81.576,34

Total na Instituição

463.167,68

567.277,02

4.854,51

0,00

562.422,51

SulAmérica Inflatie FI Renda Fixa Longo Prazo

Data Mov.

Cotização

Histórico

Cota

Quantidade

Valor Bruto

15/05/2022
25/05/2022
04/08/